In [85]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
#from patchify import patchify, unpatchify
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image

In [86]:
# Functions needed
def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

def get_features(image, model, layers):
    features = {}
    for name, layer in model._modules.items():
        image = layer(image)
        if int(name) in layers:
            features[int(name)] = image
    return features

def im_convert(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225))
    image = image + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

def save_image(tensor, path):
    image = im_convert(tensor)
    image = Image.fromarray((image * 255).astype(np.uint8))
    image.save(path)

In [87]:
# Load VGG19, instantiate constants
vgg = models.vgg19(pretrained=True).features
# Define content and style layers
content_layers = [19] # This corresponds to 'conv_4' in VGG19
style_layers = [0, 5, 10, 19, 28] # These correspond to 'conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5'
# Freeze parameters. We don't need to train any layers
for param in vgg.parameters():
    param.requires_grad_(False)
# Define transformations
transform = transforms.Compose([transforms.Resize(256), 
                                transforms.ToTensor()])
# Set content and style weighting
alpha = 1
beta = 1e6
# Define weights for style layers
style_weights = {0: 1.,
                 5: 0.75,
                 10: 0.2,
                 19: 0.2,
                 28: 0.2}





/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [88]:
#NORMALIZE IMAGE

# loader = transforms.Compose([
#     transforms.Resize((600, 600)), 
#     transforms.ToTensor(), 
# ])

style_path='/content/drive/MyDrive/Project_image_folder/style.png'

style = transform(Image.open(style_path)).unsqueeze(0)
#style=cv.resize(cv.imread(style_path), (600,600))
#cv2_imshow(style)
content_path='/content/drive/MyDrive/Project_image_folder/content.png'
content = transform(Image.open(content_path)).unsqueeze(0)
#content=cv.resize(cv.imread(content_path), (600,600))
#cv2_imshow(content)


In [89]:
#Apply VGG19
#Need PIL Images
def apply_network(content, style, optimization_iteration):
  # Compute content and style features
  content_features = get_features(content, vgg, content_layers)
  style_features = get_features(style, vgg, style_layers)

  # Compute gram matrix for each layer in style_features
  style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}
  # Initialize target image with content image
  target = content.clone().requires_grad_(True)
  # Define optimizer
  optimizer = optim.Adam([target], lr=0.003)
  # Iteration for optimization
  for i in range(1, optimization_iteration):
    target_features = get_features(target, vgg, style_layers)
    content_loss = 0
    content_loss = torch.mean((target_features[content_layers[0]] - content_features[19])**2)
    style_loss = 0
    for layer in style_weights:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        _, d, h, w = target_feature.shape
        style_gram = style_grams[layer]
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
        style_loss += layer_style_loss / (d * h * w)
    total_loss = alpha * content_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
  return target

target=apply_network(content, style, 1)


In [96]:
#Save
prework_path="/content/drive/MyDrive/Project_image_folder/prework.jpg"
save_image(target, prework_path)
prework=cv.imread(prework_path)
prework=cv.resize(prework, (600,600))
cv2_imshow(prework)

AttributeError: ignored

In [91]:
# split in patches

patch_size=3

def split_into_patches(M, N, split_image):
  return [split_image[x:x+M,y:y+N] for x in range(0,split_image.shape[0],M) for y in range(0,split_image.shape[1],N)];

style=cv.resize(cv.imread(style_path), (600,600))
content=prework
patches_style=split_into_patches(patch_size, patch_size, style)
patches_img=split_into_patches(patch_size, patch_size, content)

In [97]:
#Main?
def argmax(result_lst):
  print(result_lst)
  out=0
  max=result_lst[0]
  for i in range(len(result_lst)):
    if result_lst[i]>max:
      max=result_lst[i]
      out=i
  return out

def unite(img_patch, stl_patch):
  img_patch_pil = transform(Image.fromarray(cv.cvtColor(img_patch, cv.COLOR_BGR2RGB))).unsqueeze(0)
  stl_patch_pil = transform(Image.fromarray(cv.cvtColor(stl_patch, cv.COLOR_BGR2RGB))).unsqueeze(0)
  save_image(img_patch_pil, "/content/drive/MyDrive/Project_image_folder/some.jpg")
  pil_result=apply_network(img_patch_pil, stl_patch_pil, 1).detach().numpy()

  return cv.cvtColor(np.array(pil_result), cv.COLOR_RGB2BGR)

In [ ]:
#Formula
output=[]
for img in patches_img:
  cofficients=[]
  for stl in patches_style:
    cofficients.append(np.argmax(np.dot(img, stl)/np.dot(img, img)/np.dot(stl, stl)))
  output.append(unite(img, patches_style[argmax(cofficients)]))
output=np.array(output)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img  = Image.fromarray(output[0])
# Saving the image
img.save("/content/drive/MyDrive/Project_image_folder/work.jpg")